# Drawing the likelihood contours

Amongst the material to be presented in the final results is the contours in branching fraction space.  Here I will draw the contours based on the assumption that the pdf for the branching fractions can be approximated by a three (or four if we include the hadronic fraction) dimensional Gaussian.  

In [1]:
## imports and configuration
%cd '/home/naodell/work/wbr/analysis'
#%load_ext autoreload

from functools import partial
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from scipy.stats import multivariate_normal
from tqdm import tqdm_notebook

import scripts.plot_tools as pt
import scripts.fit_helpers as fh
from nllfit.nllfitter import ScanParameters

np.set_printoptions(precision=4)
rc_params = {
             'figure.figsize': (10, 10),
             'axes.labelsize': 20,
             'axes.facecolor': 'white',
             'axes.titlesize':'x-large',
             'legend.fontsize': 20,
             'xtick.labelsize':20,
             'ytick.labelsize':20,
             'font.size':18,
             'font.sans-serif':['Arial', 'sans-serif'],
             'mathtext.sf':'Arial',
             'lines.markersize':8.,
             'lines.linewidth':2.5,
            }
matplotlib.rcParams.update(rc_params)

%connect_info

/home/naodell/work/wbr/analysis
{
  "shell_port": 53007,
  "iopub_port": 35505,
  "stdin_port": 56221,
  "control_port": 50231,
  "hb_port": 45875,
  "ip": "127.0.0.1",
  "key": "ecd730fb-68b2e40691badaf71c3685fc",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-c9d7b1d7-1cbe-4e5a-9d54-2400f67da8eb.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [2]:
# configure, get the input data, and do any additional processing that is needed
parameters_asimov = pd.read_csv('data/model_parameters_asimov.csv', index_col='name')
corr_matrix_asimov = pd.read_csv('local_data/corr_table_full_asimov.csv', index_col='label')

parameters_unblinded = pd.read_csv('data/model_parameters_unblinded.csv', index_col='name')
corr_matrix_unblinded = pd.read_csv('local_data/corr_table_full_unblinded.csv', index_col='label')

In [3]:
# define multivariate normal distributions
val_fit_asimov = parameters_asimov.loc[['beta_e', 'beta_mu', 'beta_tau'], 'val_fit'].values
err_fit_asimov = parameters_asimov.loc[['beta_e', 'beta_mu', 'beta_tau'], 'err_fit'].values
cov_matrix_asimov = corr_matrix_asimov.iloc[:3, :3].values*np.outer(val_fit_asimov, val_fit_asimov)
f_asimov = multivariate_normal(mean=val_fit_asimov, cov=cov_matrix_asimov)

val_fit_unblinded = parameters_unblinded.loc[['beta_e', 'beta_mu', 'beta_tau'], 'val_fit'].values
err_fit_unblinded = parameters_unblinded.loc[['beta_e', 'beta_mu', 'beta_tau'], 'err_fit'].values
cov_matrix_unblinded = corr_matrix_unblinded.iloc[:3, :3].values*np.outer(val_fit_unblinded, val_fit_unblinded)
f_unblinded = multivariate_normal(mean=val_fit_unblinded, cov=cov_matrix_unblinded)

In [4]:
# plot the cost
fig, axes = plt.subplots(1, 3, figsize=(24, 10), facecolor='white', sharex = True, sharey=True)

# plot options
scan_range = (0.104, 0.114)
clvls = [0.5, 2]
    
# PDG average
from scipy.stats import multivariate_normal
mu = 0.1080
sigma = 0.0009
f_pdg = multivariate_normal(mean=[mu, mu], cov=[[sigma**2, 0.], [0., sigma**2]])

# PDG exclusive BR
pdg_e = (0.1071, 0.0016)
pdg_mu = (0.1063, 0.0015)
pdg_tau = (0.1138, 0.0021)

# electron vs. muon
scan_vals = np.linspace(scan_range[0], scan_range[1], 100)
X, Y, Z = np.meshgrid(scan_vals, scan_vals, scan_vals)
pos = np.empty(X.shape + (3,))
pos[:,:,0] = X
pos[:,:,1] = Y
pos[:,:,2] = Z

nll_pdg = f_pdg.pdf(pos[:,:,:1])

ax = axes[0]
ax.scatter([val_fit_unblinded[0]], [val_fit_unblinded[1]], marker='+', c='r')
ax.scatter([mu], [mu], marker='+', c='k')
ax.contourf(X, Y, nll_pdg, levels=clvls, linestyles='solid', colors=['y', 'g'])
#ax.contour(X, Y, cost_emu, levels=clvls, linestyles='solid', colors = 'k')

ax.fill_between(scan_range, 2*[pdg_mu[0] - pdg_mu[1], ], 2*[pdg_mu[0] + pdg_mu[1], ], facecolor='none', edgecolor='C1', hatch='/', alpha=0.5)
ax.plot(scan_range, 2*[pdg_mu[0],], color='C1', alpha=0.5)
ax.fill_betweenx(scan_range, 2*[pdg_e[0] - pdg_e[1], ], 2*[pdg_e[0] + pdg_e[1], ], facecolor='none', edgecolor='C0', hatch='\\', alpha=0.5)
ax.plot(2*[pdg_e[0],], scan_range, color='C0', alpha=0.5)

ax.set_xlabel(r'$\beta_{e}$')
ax.set_ylabel(r'$\beta_{\mu}$')
ax.set_xlim(scan_range)
ax.set_ylim(scan_range)

# muon vs. tau
#X, Y = sv_accept[:,[1, 2]].T
#X, Y = X[mask], Y[mask]
#X, Y = zoom(X.reshape(grid_size), ninter), zoom(Y.reshape(grid_size), ninter)
#
#pos[:,:,0] = X
#pos[:,:,1] = Y
#nll_pdg = f_pdg.pdf(pos)
#nll_pdg = -2*np.log(nll_pdg/nll_pdg.max())
#
#cost_mutau = zoom(cost[mask].reshape(grid_size), ninter) 
#cost_mutau -= cost_mutau.min()
#imin = cost_mutau.argmin()
#xmin = X[imin//(ninter*grid_size[0]), imin%(ninter*grid_size[0])]
#ymin = Y[imin//(ninter*grid_size[0]), imin%(ninter*grid_size[0])]
#
#ax = axes[1]
#ax.scatter([mu], [mu], marker='+', c='r')
#ax.contour(X, Y, nll_pdg, levels=clvls, linestyles='dotted', colors='r')
#ax.fill_between(scan_range, 2*[pdg_tau[0] - pdg_tau[1], ], 2*[pdg_tau[0] + pdg_tau[1], ], color='C2', hatch='/', alpha=0.1)
#ax.plot(scan_range, 2*[pdg_tau[0],], color='C2', alpha=0.5)
#ax.fill_betweenx(scan_range, 2*[pdg_mu[0] - pdg_mu[1], ], 2*[pdg_mu[0] + pdg_mu[1], ], color='C1', hatch='\\', alpha=0.1)
#ax.plot(2*[pdg_mu[0],], scan_range, color='C1', alpha=0.5)
#ax.test(0.85, 0.9, r'$\beta_{tau} = 0.108$', transform=ax.T)
#
#ax.scatter([xmin], [ymin], marker='+', c='k')
#ax.contour(X, Y, cost_mutau, levels=clvls, linestyles='solid', colors = 'k')
#ax.set_xlabel(r'$\beta_{\mu}$')
#ax.set_ylabel(r'$\beta_{\tau}$')
#
## muon vs. tau
#mask = (sv_accept[:,1] == beta_min[1])
#X, Y = sv_accept[:,[2, 0]].T
#X, Y = X[mask], Y[mask]
#X, Y = zoom(X.reshape(grid_size), ninter), zoom(Y.reshape(grid_size), ninter)
#
#pos[:,:,0] = X
#pos[:,:,1] = Y
#nll_pdg = f_pdg.pdf(pos)
#nll_pdg = -2*np.log(nll_pdg/nll_pdg.max())
#
#cost_etau = zoom(cost[mask].reshape(grid_size), ninter) 
#cost_etau -= cost_etau.min()
#imin = cost_etau.argmin()
#xmin = X[imin//(ninter*grid_size[0]), imin%(ninter*grid_size[0])]
#ymin = Y[imin//(ninter*grid_size[0]), imin%(ninter*grid_size[0])]

ax = axes[2]
#ax.scatter([mu], [mu], marker='+', c='r', label='PDG average')
#ax.contour(X, Y, nll_pdg, levels=clvls, linestyles='dotted', colors='r')
#
#ax.scatter([xmin], [ymin], marker='+', c='k', label='Asimov data')
#ax.contour(X, Y, cost_etau, levels=clvls, linestyles='solid', colors = 'k', label='2016 data')
#
#ax.plot(scan_range, 2*[pdg_e[0],], color='C0', alpha=0.5, label='_nolegend_')
#ax.fill_between(scan_range, 2*[pdg_e[0] - pdg_e[1], ], 2*[pdg_e[0] + pdg_e[1], ], edgecolor='C0', facecolor='none', hatch='/', alpha=0.5, label=r'$e$ PDG')
#
## dummy plot to add mu to legend
#ax.fill_betweenx([0., 0.], 2*[pdg_mu[0] - pdg_mu[1], ], 2*[pdg_mu[0] + pdg_mu[1], ], edgecolor='C1', facecolor='none', hatch='\\', alpha=0.5, label=r'$\mu$ PDG')
#
#ax.plot(2*[pdg_tau[0],], scan_range, color='C2', alpha=0.5, label='_nolegend_')
#ax.fill_betweenx(scan_range, 2*[pdg_tau[0] - pdg_tau[1], ], 2*[pdg_tau[0] + pdg_tau[1], ], edgecolor='C2', facecolor='none', hatch='|', alpha=0.5, label=r'$\tau$ PDG')

#ax.set_xlabel(r'$\beta_{\tau}$')
#ax.set_ylabel(r'$\beta_{e}$')
ax.legend(loc=1, framealpha=1.)

plt.tight_layout()
plt.savefig('plots/result_contours_test.pdf')
plt.savefig('plots/result_contours_test.png')
plt.show()

ValueError: could not broadcast input array from shape (100,100,100) into shape (100,100,3)

In [ ]:
fig, ax = plt.subplots(1, 1, facecolor='white')

fnorm = multivariate_normal(0, 1)
x = np.linspace(-5, 5, 1e3)
ax.plot(x, -2*np.log(fnorm.pdf(x)/fnorm.pdf(0)), 'k--')
ax.set_xlim(-5, 5)
ax.set_ylim(0, None)
ax.grid()

plt.show()